In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import chess
import chess.pgn

from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class PositionEvaluatorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(12, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x))  # Output between -1 and 1

In [ ]:
piece_to_index = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

def board_to_tensor(board):
    tensor = np.zeros((12, 8, 8), dtype=np.float32)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            idx = piece_to_index[piece.symbol()]
            row = 7 - square // 8
            col = square % 8
            tensor[idx, row, col] = 1
    return tensor


In [ ]:
from torch.utils.data import Dataset
class ChessEvalDataset(Dataset):
    def __init__(self, fen_eval_list):
        self.data = fen_eval_list  # List of (FEN, eval) tuples

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        fen, target = self.data[idx]
        board = chess.Board(fen)
        board_tensor = board_to_tensor(board)
        return torch.tensor(board_tensor), torch.tensor([target], dtype=torch.float32)


In [ ]:
def extract_fens_from_pgn(pgn_file, label_from="result"):
    positions = []
    with open(pgn_file, "r", encoding="utf-8") as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            board = game.board()
            result = game.headers.get("Result")

            # Label based on game outcome
            if result == "1-0":
                label = 1.0
            elif result == "0-1":
                label = -1.0
            else:
                label = 0.0

            # Step through moves
            for move in game.mainline_moves():
                board.push(move)
                fen = board.fen()
                positions.append((fen, label))  # ← You can also call Stockfish here if you want

    return positions


In [ ]:
NUM_EPOCHS = 100
# Example dummy data
data = [
    ("rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1", 0.2),
    ("rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1", -0.5)
]

dataset = ChessEvalDataset(data)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

model = PositionEvaluatorNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

for epoch in range(NUM_EPOCHS):
    for board_tensor, target_eval in loader:
        pred = model(board_tensor)
        loss = loss_fn(pred, target_eval)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


# Save the trained model
torch.save(model.state_dict(), 'position_evaluator_model.pth')


In [ ]:
trained_model = PositionEvaluatorNet()
trained_model.load_state_dict(torch.load('position_evaluator_model.pth'))
trained_model.eval()  # Set model to evaluation mode